**Coursebook: Understanding Embedding in LLM**

- Part 3 of Understanding Embedding in LLM
- Course Length: 9 hours
- Last Updated: July 2023
---

Developed by Algoritma's Research and Development division

## Background

The coursebook is part of the **Large Language Models Specialization** developed by [Algoritma](https://algorit.ma/). The coursebook is intended for a restricted audience only, i.e. the individuals and organizations having received this coursebook directly from the training organization. It may not be reproduced, distributed, translated or adapted in any form outside these individuals and organizations without permission.Algoritma is a data science education center based in Jakarta. We organize workshops and training programs to help working professionals and students gain mastery in various data science sub-fields: data visualization, machine learning, data modeling, statistical inference etc.

# Understanding Embedding in LLM 

## Training Objectives

- **Understanding Embeddings in Large Language Models (LLM) for Natural Language Processing**
   - Basic concepts of embeddings in LLM
   - Usage of embeddings in natural language processing
   - Demonstration of embeddings usage in text analysis

- **Advanced Embeddings in Large Language Models (LLM) for Text Processing**
   - In-depth understanding of embeddings in LLM
   - Introduction to popular embedding techniques like Word2Vec and FastText
   - Implementation of embeddings in text processing using Python
   - Demonstration of embedding techniques in text processing tasks

- **Advanced Applications of Embeddings in Text Processing with Large Language Models (LLM)**
   - Application of embeddings in advanced text processing
   - Usage of embeddings for text classification and contextual understanding
   - Demonstration of embeddings usage in advanced tasks

## Understanding Embeddings in Large Language Models (LLM) for Natural Language Processing


We have built GPT chatbot from LLM that can generate answer from our data. Let's dive deep how LLM understanding natural language by understanding Embedding. 

### Basic Concept of Embedding (Vector)

A vector (or embedding) is an array of numbers. That on its own is exciting, but what is even more exciting is that these arrays can represent more complex data like text, images, audio or even video. In the case of text, these representations are designed to capture semantic and syntactic relationships between words, allowing algorithms to understand and process language more effectively.

Word embeddings, specifically, are dense vector representations that encode the meaning of a word based on its context in a large corpus of text. In simpler terms, they map words to numerical vectors in a high-dimensional space, where similar words are located closer to each other. This is done in a vector database (we will talk about this later)

Creating these embeddings is done by an embedding model. There are multiple embedding models that can be used. OpenAI also provide embedding model but we will use free LLM model so we don't ran out of credit. We will use "all-MiniLM-L6-v2" embedding model.

Making embeddings can be visualised in the following way:

![embedding](assets/embedding.gif)

This embedding process apply in many LLM implementation, for example QnA system or GPT chatbot. The question asked to the chatbot will be embedded as well, and on the basis of similarity search, the retriever will return the embeddings with the data to answer the question. After this, the LLM will return a coherent and well-structured answer.

But let's dive deep the concept one by one start by how to perform embedding from raw text to vector form.

In [1]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\Lenovo\anaconda3\envs\langchain_test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sentences = [
    "This is document about cat",
    "This is document about car",
    "Example of the long sentences: China increased its coal-fired power capacity by 42.9 GW, or 4.5%, in the 18 months to June 2019, according to a report by Global Energy Monitor. The study also found that another 121.3 GW of coal-fired power plants are under construction in China, which has pledged to reduce its coal usage. However, the country’s absolute coal consumption has still increased in line with rising energy demand. China accounts for more than 40% of the world's total coal generation capacity."
]

In [30]:
# Perform embedding using embed_documents()
embedded_sentences = embedding_function.embed_documents(sentences)

# show embedded result
embedded_sentences[0][:25]

[0.03584069758653641,
 0.0844104215502739,
 0.004435782786458731,
 0.06132490187883377,
 -0.09679650515317917,
 -0.011024974286556244,
 -0.03996598348021507,
 0.02713777869939804,
 -0.03685332089662552,
 0.04033125191926956,
 0.022044340148568153,
 0.01601887308061123,
 -0.042161595076322556,
 0.010156717151403427,
 -0.09993438422679901,
 0.0030053777154535055,
 -0.10560218244791031,
 0.03724587708711624,
 0.013642314821481705,
 0.05051238834857941,
 0.02487064152956009,
 0.08287020027637482,
 0.05163392424583435,
 -0.014835968613624573,
 -0.0781128853559494]

In [4]:
# See len of embedded sentences
len(embedded_sentences)

3

We have three sentences and have embedded all of them. Let's see the embedding shape all of them.

In [5]:
# Shape of embedded sentences
for text in embedded_sentences:
    print(len(text))

384
384
384


In [6]:
# show the first 25 vector of first sentences
embedded_sentences[0][:25]

[0.03584069758653641,
 0.0844104215502739,
 0.004435782786458731,
 0.06132490187883377,
 -0.09679650515317917,
 -0.011024974286556244,
 -0.03996598348021507,
 0.02713777869939804,
 -0.03685332089662552,
 0.04033125191926956,
 0.022044340148568153,
 0.01601887308061123,
 -0.042161595076322556,
 0.010156717151403427,
 -0.09993438422679901,
 0.0030053777154535055,
 -0.10560218244791031,
 0.03724587708711624,
 0.013642314821481705,
 0.05051238834857941,
 0.02487064152956009,
 0.08287020027637482,
 0.05163392424583435,
 -0.014835968613624573,
 -0.0781128853559494]

We see that the embedding generates 384 vector no matter how short or long the sentences. This is because [sentence-transformers/msmarco-MiniLM-L-12-v3](sentence-transformers/msmarco-MiniLM-L-12-v3) maps sentences & paragraphs to a 384 dimensional dense vector space and ca be used for tasks like semantic search.

### Find Similarity Between Documents

We have embedding three sentences (documents) above. What if we want to know which document contains relevance information about our question?

We can resolve this problem by embedding our question to vector dimension and compute the similarity between our question and our documents.

The similarity compute using `cosine distance` which the lower the distances, the similar the vector is.

For example if we want to know which document contains information about China and coal.

In [7]:
# Embed the question/query
embed_query = embedding_function.embed_documents(['China and coal'])

In [8]:
# Import the cosine_distance
from sklearn.metrics.pairwise import cosine_distances

# Compute the cosine distance between query and documents
cosine_distances(embed_query, embedded_sentences)

array([[0.97059793, 0.92054657, 0.43505046]])

Based on result above, we know that the third document has lowest distance, that means the third document contains relevane information about "China and coal" which actually really obvious since the third document contains summary news about China coal-powered plant.

Let's see another example. What if we want to know which document contains about vehicle.

In [9]:
query = ["show document about vehicle"]
embed_query = embedding_function.embed_documents(query)

cosine_distances(embed_query, embedded_sentences)

array([[0.69098626, 0.20956952, 0.99632427]])

Since the second document contains the lowest distance, lets see the second document.

In [10]:
# show second document
sentences

['This is document about cat',
 'This is document about car',
 "Example of the long sentences: China increased its coal-fired power capacity by 42.9 GW, or 4.5%, in the 18 months to June 2019, according to a report by Global Energy Monitor. The study also found that another 121.3 GW of coal-fired power plants are under construction in China, which has pledged to reduce its coal usage. However, the country’s absolute coal consumption has still increased in line with rising energy demand. China accounts for more than 40% of the world's total coal generation capacity."]

Notice that the second document doesn't contains any "vehicle" word but contains "car" which semantically we know that car is vehicle. 

Also notice that the first and second document just have 1 different letter "cat" and "car". But the distance pretty far. 

This is because the embedding model embed the sentence based on its semantic textual meaning instead of just the letter/word like conventional embedding function.

Let's use another example

In [11]:
query = ["show document about animals"]
embed_query = embedding_function.embed_documents(query)

cosine_distances(embed_query, embedded_sentences)

array([[0.39287084, 0.61589003, 1.00654091]])

In [12]:
sentences[0]

'This is document about cat'

We see even the documents doesn't contains any "animals" word, but by compute the cosine distance of the embedding vector we can find the document that can represent the semantic meaning from our query.

## Advanced Embedding in Large Language Models (LLM) for Text Processing

### Vector Database (CHROMA DB)

When working with Large Language Models (LLMs) like GPT-4 or Google's PaLM 2, we will often be working with big amounts of unstructured, textual data. Structured data can just be stored in a SQL database, but that is much harder with unstructured data. When we for instance have a lot of text files like above example with information on a certain topic, it might be good to store this information in a different way in order to retrieve the desired data in the most efficient way. The answer to this: **Vector Databases**.

The specific vector database that we will use is the **ChromaDB** vector database.

[Chroma Website](https://docs.trychroma.com/getting-started#:~:text=Chroma%20is%20a%20database%20for,hosted%20version%20is%20coming%20soon!):

> Chroma is a database for building AI applications with embeddings. It comes with everything you need to get started built in, and runs on your machine. ChromaDB

By using `Chroma` we don't have to embed and compute the cosine distance manually.

`Chroma` and `LangChain` already integrated. Let's import the necessary function.

In [13]:
# import
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

In [32]:
# load the document and split it into chunks
loader = TextLoader("data_input/state_of_the_union.txt")
document = loader.load()

In [37]:
document

[Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citize

The document has been loaded. But if we see the document, the document contains many paragraph. Let's split the document by each paragraph so its easy for us to find which paragraph most similar to our query.

In [16]:
# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
text = text_splitter.split_documents(document)

text[:5]

[Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.', metadata={'source'

We have split each paragraph in document. Let's embed and store our sentence and embed vector into Chroma.

In [17]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(text, embedding_function)

Chroma will store our ids, embeddings, documents and metadatas into `collection`.

In [18]:
db._collection.get().keys()

dict_keys(['ids', 'embeddings', 'documents', 'metadatas'])

In [19]:
db._collection.get()['documents'][:5]

['Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.',
 'A former top litigator in private prac

In [20]:
db._collection.get()['ids'][:5]

['cc177c48-2792-11ee-bf47-c85b76361b4c',
 'cc177c68-2792-11ee-9260-c85b76361b4c',
 'cc177c4a-2792-11ee-b387-c85b76361b4c',
 'cc177c4b-2792-11ee-9ada-c85b76361b4c',
 'cc177c4c-2792-11ee-8e69-c85b76361b4c']

In [21]:
db._collection.get()['metadatas'][:5]

[{'source': 'data_input/state_of_the_union.txt'},
 {'source': 'data_input/state_of_the_union.txt'},
 {'source': 'data_input/state_of_the_union.txt'},
 {'source': 'data_input/state_of_the_union.txt'},
 {'source': 'data_input/state_of_the_union.txt'}]

Hehehe

```python
(method) def similarity_search_with_score(
    query: str,
    k: int = DEFAULT_K,
    filter: Dict[str, str] | None = None,
    **kwargs: Any
) -> List[Tuple[Document, float]]
Run similarity search with Chroma with distance.

Args:
    query (str): Query text to search for.
    k (int): Number of results to return. Defaults to 4.
    filter (Optional[Dict[str, str]]): Filter by metadata. Defaults to None.

Returns:
    List[Tuple[Document, float]]: List of documents most similar to the query text and cosine distance in float for each. Lower score represents more similarity.
```

In [22]:
# Embed query and find similar document
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search_with_score(query, k=5)

In [23]:
docs

[(Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': 'data_input/state_of_the_union.txt'}),
  1.1972053050994873),
 (Document(page_content='A former top litigator in private practice. A former federal public defe

We see the `docs` return 5 most similar document.

What if we have bunch of txt file in one folder and want to embed all of them. We can load all files in document using `DirectoryLoader` and split each paragraph using `RecursiveCharacterTextSplitter`.

In [24]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [25]:
# Load and process the all text files in new_articles
loader = DirectoryLoader('data_input/new_articles/', glob="./*.txt", loader_cls=TextLoader)

documents = loader.load()

Because we have bunch of text file, we have to perform text splitting using `RecursiveCharacterTextSplitter`.

In [26]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text = text_splitter.split_documents(documents)

In [27]:
text[:5]

[Document(page_content='Welcome to The Interchange! If you received this in your inbox, thank you for signing up and your vote of confidence. If you’re reading this as a post on our site, sign up here so you can receive it directly in the future. Every week, we’ll take a look at the hottest fintech news of the previous week. This will include everything from funding rounds to trends to an analysis of a particular space to hot takes on a particular company or phenomenon. There’s a lot of fintech news out there and it’s our job to stay on top of it — and make sense of it — so you can stay in the know. — Mary Ann and Christine\n\nBusy, busy, busy\n\nIt was a busy week in startup and venture lands, and the fintech space was no exception.', metadata={'source': 'data_input\\new_articles\\05-07-fintech-space-continues-to-be-competitive-and-drama-filled.txt'}),
 Document(page_content='Busy, busy, busy\n\nIt was a busy week in startup and venture lands, and the fintech space was no exception.\n

We have split all our text. The following steps is same like the code before.

In [28]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
vectordb = Chroma.from_documents(text, embedding_function)

# query it
query = "What is the news about Pando?"
docs = vectordb.similarity_search_with_score(query)

docs

[(Document(page_content='Pando was co-launched by Jayakrishnan and Abhijeet Manohar, who previously worked together at iDelivery, an India-based freight tech marketplace — and their first startup. The two saw firsthand manufacturers, distributors and retailers were struggling with legacy tech and point solutions to understand, optimize and manage their global logistics operations — or at least, that’s the story Jayakrishnan tells.\n\n“Supply chain leaders were trying to build their own tech and throwing people at the problem,” he said. “This caught our attention — we spent months talking to and building for enterprise users at warehouses, factories, freight yards and ports and eventually, in 2018, decided to start Pando to solve for global logistics through a software-as-a-service platform offering.”', metadata={'source': 'data_input\\new_articles\\05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt'}),
  0.9159137606620789),
 (Document(page_content='Signaling that inv

### Saving to Disk

Embedding text takes much to time and resources to processes, especially if we have large amount of text.

To encounter this problem, we can save our vector database (including the vector sentences) to disk so each time we need to use this vector we just load the saved vector database.

To save vector db to disk, simply initialize the Chroma client and pass the directory where you want the data to be saved to.

> Caution: Chroma makes a best-effort to automatically save data to disk, however multiple in-memory clients can stomp each other's work. As a best practice, only have one client per path running at any given time.

> Protip: Sometimes you can call db.persist() to force a save

In [29]:
# save to disk
# load it into Chroma
vectordb = Chroma.from_documents(text, embedding_function, persist_directory="./chroma_db")
vectordb.persist()

The vector db will save to ./chroma_db folder. So whenever we want to use the embedding file. Just load it from ".chroma_db" file.

Let's see how to load saved vector db in Chroma.

In [ ]:
vectordb_load = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)

# query it
query = "What is the news about Pando?"
docs_load = vectordb_load.similarity_search_with_score(query)

docs_load

[(Document(page_content='Pando was co-launched by Jayakrishnan and Abhijeet Manohar, who previously worked together at iDelivery, an India-based freight tech marketplace — and their first startup. The two saw firsthand manufacturers, distributors and retailers were struggling with legacy tech and point solutions to understand, optimize and manage their global logistics operations — or at least, that’s the story Jayakrishnan tells.\n\n“Supply chain leaders were trying to build their own tech and throwing people at the problem,” he said. “This caught our attention — we spent months talking to and building for enterprise users at warehouses, factories, freight yards and ports and eventually, in 2018, decided to start Pando to solve for global logistics through a software-as-a-service platform offering.”', metadata={'source': 'data_input\\new_articles\\05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt'}),
  0.9159137606620789),
 (Document(page_content='Signaling that inv

After we load the vector db, we can still perform query semantic search and got the same result from our vector database before saving to disk.

------------ BORDER ----------------

## Workflow: Applications of Embedding in Text Processing with Large Language Models (LLM)

### Create QnA System from Vector Database

In this section, we will explain the workflow to create a Q&A system by leveraging a vector database and the power of LangChain and Chroma DB. This workflow will serve as a guide for building similar projects and can be adapted for various text processing tasks, including summarized text analysis, document retrieval, and more.

#### Business Problem

#### Data Preparation

#### Embedding and Vector Database Creation

In [38]:
# embeddingnya ringan 80mb
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

vectordb_load = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)

#### Query Processing

In [39]:
# Ask a question using the QA chain
question = "What is Pando vision?"
similar_docs = vectordb_load.similarity_search(question)

In [40]:
similar_docs

[Document(page_content='Pando was co-launched by Jayakrishnan and Abhijeet Manohar, who previously worked together at iDelivery, an India-based freight tech marketplace — and their first startup. The two saw firsthand manufacturers, distributors and retailers were struggling with legacy tech and point solutions to understand, optimize and manage their global logistics operations — or at least, that’s the story Jayakrishnan tells.\n\n“Supply chain leaders were trying to build their own tech and throwing people at the problem,” he said. “This caught our attention — we spent months talking to and building for enterprise users at warehouses, factories, freight yards and ports and eventually, in 2018, decided to start Pando to solve for global logistics through a software-as-a-service platform offering.”', metadata={'source': 'data_input\\new_articles\\05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt'}),
 Document(page_content='Pando was co-launched by Jayakrishnan and A

#### Q&A System Implementation

In [41]:
from dotenv import load_dotenv
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub

# Load HuggingFace token from env
load_dotenv()

# Load the LLM and create a QA chain
llm = HuggingFaceHub(repo_id="declare-lab/flan-alpaca-large",
                     model_kwargs={"temperature":0.9, "max_length":512})

# Create qa chain
qa_chain = load_qa_chain(llm, chain_type="stuff")

#### Display Results

In [42]:
# Generate answer from related document (from similarity search result)
response = qa_chain.run(input_documents=similar_docs, question=question)

response

'The Pando vision is to provide a platform that helps businesses optimize and manage their global logistics operations.'

#### Chaining 

**Retriever**

In order to get the data out of the database again, we need to create a retriever. This retriever will return all the documents (or chucks) related to the question asked:

In [43]:
# Create retriever
retriever = vectordb_load.as_retriever()

**RetrievalQA**

In [44]:
from langchain.chains import RetrievalQA

# create the chain to answer questions
# so we can cut the process/code to generate answer from related document
qa_chain = RetrievalQA.from_chain_type(llm = llm,
                                  chain_type = "stuff",
                                  retriever = retriever,
                                  return_source_documents = True,
                                  verbose = True)

In [45]:
# We see the same result from our previusly QnA system. 
# But more simply because we "chain"ing the process using LangChain Chain feature
# cool tho
qa_chain("what is Pando Vision?")



> Entering new  chain...



> Finished chain.


{'query': 'what is Pando Vision?',
 'result': 'Pando Vision is a software-as-a-service platform that helps businesses optimize and manage their global logistics operations. It uses algorithms and forms of machine learning to make predictions around supply chain events.',
 'source_documents': [Document(page_content='Pando was co-launched by Jayakrishnan and Abhijeet Manohar, who previously worked together at iDelivery, an India-based freight tech marketplace — and their first startup. The two saw firsthand manufacturers, distributors and retailers were struggling with legacy tech and point solutions to understand, optimize and manage their global logistics operations — or at least, that’s the story Jayakrishnan tells.\n\n“Supply chain leaders were trying to build their own tech and throwing people at the problem,” he said. “This caught our attention — we spent months talking to and building for enterprise users at warehouses, factories, freight yards and ports and eventually, in 2018, d

### Create QnA System From PDF Source

#### Business Problem

#### Data Preparation

In [46]:
from langchain.document_loaders import UnstructuredPDFLoader 

def loadPDFFromLocal(pdf_file_path="data_input/Eurovision_Song_Contest_2023.pdf"):
    loader = UnstructuredPDFLoader(pdf_file_path)
    loaded_docs = loader.load()
    return loaded_docs

[source MoM](https://www2.deloitte.com/content/dam/Deloitte/au/Documents/finance/insolvency/virgin/deloitte-au-fa-virgin-australia-minutes-to-6th-coI-meeting-10-august-2020-010920.pdf)

In [47]:
from langchain.text_splitter import CharacterTextSplitter

# Example of minutes of meeting
document = loadPDFFromLocal("data_input/deloitte-au-fa-virgin-australia-minutes.pdf")
# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
text = text_splitter.split_documents(document)

Created a chunk of size 1310, which is longer than the specified 1000


In [48]:
text[:5]

[Document(page_content='Australian Securities & Investments Commission\n\nElectronic Lodgement\n\nDocument No. 7EAZ97262\n\nLodgement date/time: 31-08-2020 13:56:41\n\nReference Id: 143848313\n\nForm 5011 Corporations Act 2001 s436E, 439A Insolvency Practice Rules (Corporations) 2016 s75-145(1)(c)\n\nCopy of minutes of meeting\n\nLiquidator details\n\nRegistered liquidator number\n\n326810\n\nRegistered liquidator name\n\nRICHARD JOHN HUGHES\n\nCompany details\n\nCompany name\n\nVIRGIN AUSTRALIA HOLDINGS LIMITED\n\n100 686 226\n\nSection under which minutes are lodged\n\nDate of meeting\n\n10-08-2020\n\nAre the minutes being lodged for a meeting convened under s436E, or s439A of the Corporations Act 2001?\n\nNo\n\nCertification\n\nI certify that the attached minutes of meeting are a true copy of the original minutes of meeting signed by the chair of the meeting as identified in the minutes.\n\nYes\n\nASIC Form 5011 Ref 143848313\n\nPage 1 of 2\n\nForm 5011 - Copy of minutes of meeting 

#### Embedding and Vector Database Creation

In [49]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
vectordb_pdf = Chroma.from_documents(text, embedding_function, persist_directory="./chroma_db_pdf")
vectordb_pdf.persist()

In [50]:
# if want to load again from disk
vectordb_pdf = Chroma(persist_directory="./chroma_db_pdf", embedding_function=embedding_function)

#### Q&A System Implementation

In [51]:
# Create retriever
retriever = vectordb_pdf.as_retriever()

In [52]:
from langchain.chains import RetrievalQA

# create the chain to answer questions
# so we can cut the process/code to generate answer from related document
qa_chain_pdf = RetrievalQA.from_chain_type(llm = llm,
                                  chain_type = "stuff",
                                  retriever = retriever,
                                  return_source_documents = True,
                                  verbose = True)

In [53]:
qa_chain_pdf("Who is the chairperson of the meeting?")



> Entering new  chain...



> Finished chain.


{'query': 'Who is the chairperson of the meeting?',
 'result': 'The chairperson is the person who is responsible for coordinating the meeting.',
 'source_documents': [Document(page_content='TIME AND PLACE OF MEETING The Chairperson stated he had not received any objections to the time and place of the meeting. Therefore, pursuant to 75-30 of the IPR, he declared that the meeting to be held at a time and place convenient for the majority of persons entitled to receive notice of the meeting.\n\nVA-E-115\n\nVOTING ON RESOLUTIONS\n\nThe Chairperson noted that ordinarily, motions put to the COI would be resolved on the voices. As the meeting was being conducted virtually and given the fee resolutions being put to the committee, voting on these resolutions would be conducted by online poll to ensure there was a clear record of votes. He advised the link to access the online voting form would be circulated to all committee members in attendance by email during the meeting.\n\nAGENDA\n\nThe Ch

#### Display Results

In [54]:
qa_chain_pdf("Who is the chairperson of the meeting?")



> Entering new  chain...

> Finished chain.


{'query': 'Who is the chairperson of the meeting?',
 'result': 'The chairperson is the person who is responsible for coordinating the meeting.',
 'source_documents': [Document(page_content='TIME AND PLACE OF MEETING The Chairperson stated he had not received any objections to the time and place of the meeting. Therefore, pursuant to 75-30 of the IPR, he declared that the meeting to be held at a time and place convenient for the majority of persons entitled to receive notice of the meeting.\n\nVA-E-115\n\nVOTING ON RESOLUTIONS\n\nThe Chairperson noted that ordinarily, motions put to the COI would be resolved on the voices. As the meeting was being conducted virtually and given the fee resolutions being put to the committee, voting on these resolutions would be conducted by online poll to ensure there was a clear record of votes. He advised the link to access the online voting form would be circulated to all committee members in attendance by email during the meeting.\n\nAGENDA\n\nThe Ch

In [55]:
qa_chain_pdf("Who is the chairperson of the meeting?")



> Entering new  chain...



> Finished chain.


{'query': 'Who is the chairperson of the meeting?',
 'result': 'The chairperson is the person who is responsible for coordinating the meeting.',
 'source_documents': [Document(page_content='TIME AND PLACE OF MEETING The Chairperson stated he had not received any objections to the time and place of the meeting. Therefore, pursuant to 75-30 of the IPR, he declared that the meeting to be held at a time and place convenient for the majority of persons entitled to receive notice of the meeting.\n\nVA-E-115\n\nVOTING ON RESOLUTIONS\n\nThe Chairperson noted that ordinarily, motions put to the COI would be resolved on the voices. As the meeting was being conducted virtually and given the fee resolutions being put to the committee, voting on these resolutions would be conducted by online poll to ensure there was a clear record of votes. He advised the link to access the online voting form would be circulated to all committee members in attendance by email during the meeting.\n\nAGENDA\n\nThe Ch

In [56]:
answer = qa_chain_pdf("What is the agenda of the meeting?")

answer



> Entering new  chain...

> Finished chain.


{'query': 'What is the agenda of the meeting?',
 'result': 'The agenda of the meeting is to fix or determine the remuneration of the Administrators.',
 'source_documents': [Document(page_content='RECORDING OF MEETING\n\nThe Chairperson advised he intention to record the meeting for the purpose of minute taking. He asked if any committee members objected to the meeting being recorded. No objections to the recording of the meeting were made.\n\nQUORUM\n\nThe Chairperson declared a quorum present at the meeting as a majority of the committee members were seen as viewing the online meeting platform at that time.\n\nTABLING OF DOCUMENTS\n\nThe Chairperson tabled the following documents: ▪ Notice of the Meeting of the Committee of Inspection dated 3 August\n\n2020 Remuneration Approval Report dated 3 August 2020 (Remuneration Report).\n\n▪\n\nThe Chairperson advised the notice of the meeting and Remuneration Report were circulated to all members by email on 3 August 2020 and he would conside

In [57]:
answer['result']

'The agenda of the meeting is to fix or determine the remuneration of the Administrators.'

In [58]:
qa_chain_pdf("What is the resolution of the meeting?")



> Entering new  chain...



> Finished chain.


{'query': 'What is the resolution of the meeting?',
 'result': 'The resolution of the meeting is to fix or determine the remuneration of the Administrators.',
 'source_documents': [Document(page_content='TIME AND PLACE OF MEETING The Chairperson stated he had not received any objections to the time and place of the meeting. Therefore, pursuant to 75-30 of the IPR, he declared that the meeting to be held at a time and place convenient for the majority of persons entitled to receive notice of the meeting.\n\nVA-E-115\n\nVOTING ON RESOLUTIONS\n\nThe Chairperson noted that ordinarily, motions put to the COI would be resolved on the voices. As the meeting was being conducted virtually and given the fee resolutions being put to the committee, voting on these resolutions would be conducted by online poll to ensure there was a clear record of votes. He advised the link to access the online voting form would be circulated to all committee members in attendance by email during the meeting.\n\nAG

In [59]:
qa_chain_pdf("What's the action plan of to fix or determine the remuneration of the administrators?")



> Entering new  chain...



> Finished chain.


{'query': "What's the action plan of to fix or determine the remuneration of the administrators?",
 'result': 'The action plan to fix or determine the remuneration of the administrators is to determine the sum equal to the cost of time spent by the Administrators and their partners and staff, calculate the hourly rates as detailed in the Initial Remuneration Notice of 21 April 2020, up to a capped amount of $13,383,765.00 exclusive of GST as detailed in schedules 2.1 to 2.50 in the Remuneration Approval Report dated 3 August 2020, and that the Administrators can draw the remuneration on a monthly basis or as required.',
 'source_documents': [Document(page_content='“That the future remuneration of the Administrators for the period 1 July 2020 to the completion of the Voluntary Administration is determined at a sum equal to the cost of time spent by the Administrators and their partners and staff, calculated at the hourly rates as detailed in the Initial Remuneration Notice of 21 April 2

### Dive Deeper

In [62]:
# Example of BI financial statements
document_bi = loadPDFFromLocal("data_input/BAB-II-Kinerja-dan-Prospek-Ekonomi-Nasional-Optimis-dan-Waspada.pdf")
# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
text_bi = text_splitter.split_documents(document_bi)

Created a chunk of size 1285, which is longer than the specified 1000
Created a chunk of size 1126, which is longer than the specified 1000
Created a chunk of size 1072, which is longer than the specified 1000
Created a chunk of size 1009, which is longer than the specified 1000
Created a chunk of size 1325, which is longer than the specified 1000
Created a chunk of size 1655, which is longer than the specified 1000
Created a chunk of size 1331, which is longer than the specified 1000
Created a chunk of size 1552, which is longer than the specified 1000
Created a chunk of size 1654, which is longer than the specified 1000
Created a chunk of size 1019, which is longer than the specified 1000
Created a chunk of size 1033, which is longer than the specified 1000
Created a chunk of size 1505, which is longer than the specified 1000
Created a chunk of size 1072, which is longer than the specified 1000
Created a chunk of size 1063, which is longer than the specified 1000
Created a chunk of s

In [ ]:
text_bi[:5]

[Document(page_content='BAB II\n\nKINERJA DAN PROSPEK EKONOMI NASIONAL: OPTIMIS DAN WASPADA\n\nSinergi kebijakan Bank Indonesia, Pemerintah, dan Komite Stabilitas Sistem Keuangan (KSSK) dapat menopang berlanjutnya proses pemulihan perekonomian nasional pada 2022, di tengah kondisi perekonomian global yang menurun. Pertumbuhan ekonomi pada 2022 diprakirakan bias ke atas dalam kisaran proyeksi Bank Indonesia pada 4,5-5,3% serta didukung stabilitas moneter dan stabilitas sistem keuangan yang tetap terjaga. Pertumbuhan ekonomi pada 2023 diprakirakan tetap kuat, meskipun sedikit melambat ke titik tengah kisaran 4,5-5,3% sejalan penurunan prospek pertumbuhan ekonomi dunia, sebelum kemudian meningkat kembali menjadi 4,7-5,5% pada 2024.', metadata={'source': 'data_input/BAB-II-Kinerja-dan-Prospek-Ekonomi-Nasional-Optimis-dan-Waspada.pdf\u200b.pdf'}),
 Document(page_content='dalam kisaran 4,5-5,3%, meskipun sedikit melambat dipengaruhi perlambatan ekonomi global. Stabilitas eksternal tetap terj

In [ ]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
vectordb_BI = Chroma.from_documents(text_bi, embedding_function, persist_directory="./chroma_db_BI")
vectordb_BI.persist()

In [ ]:
vectordb_BI = Chroma(persist_directory="./chroma_db_BI", embedding_function=embedding_function)

In [ ]:
# Create retriever
retriever = vectordb_BI.as_retriever()

In [ ]:
from langchain.chains import RetrievalQA

# create the chain to answer questions
# so we can cut the process/code to generate answer from related document
qa_chain_BI = RetrievalQA.from_chain_type(llm = llm,
                                  chain_type = "stuff",
                                  retriever = retriever,
                                  return_source_documents = True,
                                  verbose = True)

In [ ]:
qa_chain_BI("How is the prospect of Lapangan Usaha (LU)?")



> Entering new  chain...



> Finished chain.


{'query': 'How is the prospect of Lapangan Usaha (LU)?',
 'result': 'The prospect of Lapangan Usaha (LU) is positive.',
 'source_documents': [Document(page_content="83\n\nLaporan Keuangan Tahunan Bank Indonesia Tahun 2022 Bank Indonesia Annual Financial Statements 2022\n\nor IDR2,412,590 million as at December 31, 2021.\n\nsebesar SDR124,030,857.39 atau setara dengan Rp2.573.697\n\njuta, meningkat dari SDR120,729,997.80 atau Rp2.412.590\n\njuta pada 31 Desember 2021.\n\nD.5.1.2.\n\nInternational Islamic Liquidity Management\n\nD.5.1.2.\n\nInternational Islamic Liquidity Management\n\n(IILM)\n\n(IILM)\n\nIILM's objective is to provide high-quality, liquid, and\n\nTujuan dibentuknya IILM adalah untuk menyediakan\n\nhigh-rating internationally tradable short-term sharia-\n\ninstrumen keuangan syariah jangka pendek yang berkualitas\n\nbased financial instruments used mainly to support liquidity\n\ntinggi, likuid dan dapat diperdagangkan secara internasional\n\nmanagement by Islamic financi

In [ ]:
qa_chain_BI("Why the prospect of Lapangan Usaha (LU) is positive?")



> Entering new  chain...

> Finished chain.


{'query': 'Why the prospect of Lapangan Usaha (LU) is positive?',
 'result': 'The prospect of Lapangan Usaha (LU) is positive because it shows that the Bank Indonesia is taking steps to reduce the risk of a financial crisis. This is due to the fact that the Bank Indonesia has adopted a liquidity policy that is aimed at reducing the risk of a financial crisis. This policy is based on the principle of balancing the risk of a financial crisis with the benefit of reducing the risk of a financial crisis.',
 'source_documents': [Document(page_content="83\n\nLaporan Keuangan Tahunan Bank Indonesia Tahun 2022 Bank Indonesia Annual Financial Statements 2022\n\nor IDR2,412,590 million as at December 31, 2021.\n\nsebesar SDR124,030,857.39 atau setara dengan Rp2.573.697\n\njuta, meningkat dari SDR120,729,997.80 atau Rp2.412.590\n\njuta pada 31 Desember 2021.\n\nD.5.1.2.\n\nInternational Islamic Liquidity Management\n\nD.5.1.2.\n\nInternational Islamic Liquidity Management\n\n(IILM)\n\n(IILM)\n\nII